In [ ]:
%run -i ../python/common.py


closeAllOpenTtySessions()
bash = BashSession()


# remember to avoid something having a history entry put a space at the beginning of the line
#bash.run(" history")

# The Shell - Part II: Some UNIX details

<!-- produced by: 
htmlFig("[{'src': '../images/UnixRunning.png'}]", 
        id="fig:runningunix", 
        align="right", 
        width="50%",
        margin="auto auto auto 1.5em",
        caption="Figure: Running Unix system.", 
        captionalign="left")
-->
<table id="fig:runningunix" align="right" width="50%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: auto auto auto 1.5em">
    <tr style="padding: 0; margin: 0;"> 
        <td colspan="1" width="100%" style="padding: 0; margin: 0 0 0 0; background-color:inherit;">
            <div style="padding: 0; margin: 0 0 0 0;">
              <figure style="padding: 0; margin: 0 0 0 0; width:100%;">
                   <img src="../images/UnixRunning.png" width="100%" style="padding: 0; margin: 0;">
                </figure>
            </div>
        </td>
    </tr>
    <caption align="bottom" style="text-align: left; padding: 0; margin: 0;" >
          <i>Figure: Running Unix system.</i> 
    </caption>
</table>
At this point we should have an understanding of how a running UNIX system breaks down into the Operating System (OS) kernel and user processes. The way the running software of computers system is organized and interacts is often called the "runtime" organization or structure.   Bellow is a brief recap of what we know so far and various associated terms, if any of this does not make sense please go back and review the prior chapters.

## Brief Recap
The OS **kernel** provides a core set of operations, **system calls**, that permit a currently running user program to work with various types of files and to start new running programs, **processes**, from **binary executable** files.  The UNIX systems provide a **terminal** orient, **shell** program that reads and executes  **command line** .  The shell can serve as a human-oriented, interactive interface, to the capabilities of the kernel.  A core feature of the shell is its ability to located and run executable files as **external commands**.   When we create new terminal windows, the UNIX OS starts a new independent shell process, whose **standard streams**, are connected to the terminal via a special **device file** type.  In this way, each new terminal gets us a new shell process that we can use to execute command lines.  

To provide additional flexibility and control the shell goes through several steps to process a command line.  This includes nine different **expansions**.  As part the command line syntax of the shell it provides us with the ability to control the Input/Output (I/O) of the command to be run, in the form of **re-directions**.  More generally, the UNIX I/O model, is one of "files" and "streams".  The kernel provides operations such as **open** that allows a process to attach an existing, or new file, as a stream.  A process can have many streams attached to it.  The kernel maintains a **file descriptor table**, for each process, the entries track streams attached to the process.  

A process uses an numeric index of file descriptor table entry to **read** bytes from or **write** bytes to a particular stream.  In turn, the bytes will come from or go to the file that the stream connects the process too.  The numeric indexes are often called **file descriptors**.  The file descriptors, **0 - standard input**, **1 - standard output**, and **2 - standard error** have special meaning as UNIX programs will by default use these as there source of data and where their normal and error data is sent.  Various re-directions operators let's us setup the streams that a command will have open before it begins to run.  

The kernel provides a special, **pipe**, file type. A Pipe allows two processes to communicate in a coordinated way.  Where one process can write to the pipe and another can read from the pipe.  The kernel takes care of the details to ensure that the two processes synchronize correctly so that no data is lost and the processes will wait as needed.  Exploiting pipes the shell provides the ability to create **pipelines** where several commands can be chained together.  The chain is formed from left to right  with the `|` symbol separating the individual simple commands.  The standard output of the left command is setup to the standard input of the right command.

Using the abilities of the kernel the shell lets us not only run external commands as  **Synchronous**, **child processes** of the shell,  but also **Asynchronously**.  By default, when we run external commands, the shell will wait for the newly started child process to finish before it goes on to read the next command line to work on.  However, using the background operator, `&`, we can tell the shell not to wait, but rather start the command as a background child process and immediately continue to the next command line.   

Various information is provided by the kernel and shell to new processes that are started. **Exported** shell variables,  "name=value" pairs,  called environment variables, are passed to newly created processes in addition to any command line arguments specified by the command line.  Copies of the open file descriptors and the **current working directory** of the new processes are inherited from the parent shell's values.   

When processes exit, they provide an **exit status** value, in the form of an integer, to the kernel.  The shell, as a parent of the child processes, it starts to run the external commands, the shell gathers the exit statuses.  The UNIX convention, is that an exit status of 0 means success and a non-zero exit status means some form of failure.  The shell lets us examine and use the exit status of external command to create various programming structures such as conditionals, if statements and loops.

Finally, in addition to supporting interactive use, where the shell waits for the next command line to be entered by us at the terminal, the shell program  can be started with an argument that is a **shell script** file from which it will read the command lines to execute.  

## Overview

In this chapters we will focus on rounding out our understanding of the core concepts that make up how we organize the running software of a computer system.  Using UNIX as our example we will introduce the idea of users, privilege, permissions and credentials. We will then revisit Files and Streams along with Processes.  As we go along we will both introduce shell commands that let related to the topics and we will expand our view of what is going on in the kernel.  Finally we conclude by putting all the pieces together to provide a more detailed view of a running UNIX system. 

## Multi-User Operating Systems

One of the defining features of UNIX is its origin as an operating system that allows multiple people to "securely" share and use a single computer at the same time.  UNIX's popularity as the operating system used on the server computers of the internet have put its model and mechanisms for sharing a computer securely, at center of modern computer security.  Much of what we hear about on the news and see in movies when it comes to security will often involve UNIX systems in one way or another. 

Our goals in this section is to introduce the core terms and ideas that extend the idea of having multiple running programs to having the processes associated with different users.

### Multi-programming

As we have seen on UNIX, and are familiar with on most of our "devices", we take it for granted that modern computer run [several programs at the same time](../unix/shellintro.ipynb#sec:processcreation).  While it is more rare today this is not the only runtime organization that is used.  In the past, operating systems for personal computers, such as [DOS](https://en.wikipedia.org/wiki/DOS) focused on running a single program at a time.  The software that forms the operating system "kernel" for such single program runtimes are generally much simpler as well as the hardware features required.  To this end when their is no need for multiple processes like on simple computers that are embedded on devices like smart light bulbs or buttons. 

#### Multi-Processing

Runtimes that are designed to run more than one program at a time are generally referred to as multi-processing systems.  Typically on such systems several running programs can be started at the same time. Beyond the basic ability to start several, concurrently running programs, systems can vary in a great deal as to the level of transparency and independence the running programs have with respect to each other.  Clearly, UNIX is a multi-processing system.  UNIX built around the notion that a process should largely be able to ignore that it is potentially sharing the computers with other processes.  As a matter of fact, a program written to run on UNIX can be developed as if it was running on the computer alone along with the kernel.  It is the UNIX kernel's job to ensure that this illusion of isolated use and independence is maintained if the program wants it.






### Multi-user

#### Split, two-level, security model

##### Privileged (Kernel)

##### Non-privileged (User Processes)

### UNIX users and groups

#### UIDs and GIDs

#### whoami and id

#### /etc/passwd and /etc/group

#### The `root` user

### Kernel View

- table of authorized users and groups

#### Traditional: User Group Credential Model

#### Modern: Capabilities


## Revisiting UNIX Files and Streams

### Types of Files

### Meta data

#### Ownership and Permissions

#### Operations and Permissions

### Kernel View

- file as objects in kernel 
- regular and special 


## Revisiting UNIX Processes

### Process Management Commands

#### Background Commands

##### Jobs and Processes

##### Signals

### Kernel View


### Putting it all together

#### Pipes and Pipelines
